In [8]:
# %load_ext nb_black

In [26]:
import os
import json
import logging
from datetime import datetime

In [27]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

In [28]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [29]:
sess = sagemaker.Session()
region = sess.boto_region_name
print(region)

us-east-1


In [30]:
# role_arn = sagemaker.get_execution_role()
# role_arn = os.getenv("SGMKR_ROLE_ARN")
role_arn = "arn:aws:iam::381492146542:role/service-role/AmazonSageMaker-ExecutionRole-20240215T024797"

In [31]:
bucket_name = "scarecrow-pest-data"
data_path = "prepare_data"

nclasses = 6
nimgs_train = 280
nepochs = 50
mini_batch_size = 10

train_instance_type = "ml.g4dn.2xlarge"
job_name_prefix = "pest-clf-ib-resent-"

In [32]:
train_image_uri = sagemaker.image_uris.retrieve(
    framework="image-classification",
    region=region,
    image_scope="training",
    version="latest",
)
print(train_image_uri)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


In [33]:
s3_output_path = "s3://{}/{}/{}".format(bucket_name, data_path, "model_output")

In [34]:
clf_estimator = sagemaker.estimator.Estimator(
    image_uri=train_image_uri,
    role=role_arn,
    instance_count=1,
    instance_type=train_instance_type,
    volume_size=50,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_path,
    sagemaker_session=sess,
)

In [35]:
clf_estimator.set_hyperparameters(
    num_classes=nclasses,  # update this
    epochs=nepochs,  # update this
    num_training_samples=nimgs_train,  # update this
    mini_batch_size=mini_batch_size,  # update this
    num_layers=18,
    use_pretrained_model=1,
    image_shape="3,224,224",
    resize=256,
    learning_rate=0.001,
    use_weighted_loss=1,
    augmentation_type="crop_color_transform",
    precision_dtype="float32",
    multi_label=0,
)

In [36]:
s3_train_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "train_imgs")
s3_valid_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_imgs")
s3_train_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "train_annots")
s3_valid_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_annots")

train_imgs = sagemaker.inputs.TrainingInput(
    s3_train_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_imgs = sagemaker.inputs.TrainingInput(
    s3_valid_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
train_annot = sagemaker.inputs.TrainingInput(
    s3_train_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_annot = sagemaker.inputs.TrainingInput(
    s3_valid_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)

data_channels = {
    "train": train_imgs,
    "validation": valid_imgs,
    "train_lst": train_annot,
    "validation_lst": valid_annot,
}

In [37]:
timestamp = (
    str(datetime.now().replace(microsecond=0)).replace(" ", "-").replace(":", "-")
)
job_name = job_name_prefix + timestamp
print(job_name)

pest-clf-ib-resent-2024-02-15-14-47-56


In [38]:
clf_estimator.fit(inputs=data_channels, logs=True, job_name=job_name)

INFO:sagemaker:Creating training-job with name: pest-clf-ib-resent-2024-02-15-14-47-56


2024-02-15 13:48:00 Starting - Starting the training job...
2024-02-15 13:48:15 Starting - Preparing the instances for training....
2024-02-15 13:49:46 Downloading - Downloading the training image...........
2024-02-15 13:52:57 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Thu Feb 15 13:53:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                  

ClientError: An error occurred (InvalidSignatureException) when calling the GetLogEvents operation: Signature expired: 20240215T135723Z is now earlier than 20240215T140014Z (20240215T140514Z - 5 min.)

In [45]:
infer_instance_type = "ml.t2.medium"
model_name = job_name
endpoint_name = job_name

In [46]:
clf_predictor = clf_estimator.deploy(
    initial_instance_count=1,
    instance_type=infer_instance_type,
    endpoint_name=endpoint_name,
    model_name=model_name,
)

INFO:sagemaker:Creating model with name: pest-clf-ib-resent-2024-02-15-14-47-56
INFO:sagemaker:Creating endpoint-config with name pest-clf-ib-resent-2024-02-15-14-47-56
INFO:sagemaker:Creating endpoint with name pest-clf-ib-resent-2024-02-15-14-47-56


-----------!

In [53]:
sgmkr_runt = boto3.client("runtime.sagemaker")

In [58]:
with open("./image/1862.jpg", "rb") as image:
        payload = image.read()
        payload = bytearray(payload)
        
# print(payload)   
       
response = sgmkr_runt.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType = 'image/jpeg',
    Accept = "application/json;verbose",
    Body = payload,
)

prediction = json.loads(response['Body'].read().decode())
print(prediction)


bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x02\xd0\x05\x00\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x9

In [44]:
clf_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: pest-clf-ib-resent-2024-02-15-14-47-56
INFO:sagemaker:Deleting endpoint with name: pest-clf-ib-resent-2024-02-15-14-47-56
